### Modify the file path according to your environment

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'CV_HW3/assignment3/'
FOLDERNAME = 'cvhw3'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# This downloads the COCO dataset to your Drive
# if it doesn't already exist.
# %cd /content/drive/MyDrive/cvhw3/CV/datasets
# !bash get_datasets.sh

%cd /content/drive/MyDrive/cvhw3

ValueError: mount failed

# Image Captioning with Transformers
You have now implemented a vanilla RNN and for the task of image captioning. In this notebook you will implement key pieces of a transformer decoder to accomplish the same task.

**NOTE:** This notebook will be primarily written in PyTorch rather than NumPy, unlike the RNN notebook.

In [ ]:
# Setup cell.
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from CV.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from CV.transformer_layers import *
from CV.captioning_solver_transformer import CaptioningSolverTransformer
from CV.classifiers.transformer import CaptioningTransformer
from CV.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from CV.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # Set default size of plots.
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# COCO Dataset
As in the previous notebooks, we will use the COCO dataset for captioning.

In [ ]:
import torch
import torch.optim as optim

def load_checkpoint(model, optimizer, file_path):
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss_history = checkpoint['loss_history']
    print(f"Checkpoint loaded: epoch {epoch}")
    return model, optimizer, epoch, loss_history

data = load_coco_data(pca_features=False)
torch.manual_seed(231)
np.random.seed(231)

# 모델과 옵티마이저 초기화
transformer = CaptioningTransformer(
        word_to_idx=data['word_to_idx'],
        input_dim=data['train_features'].shape[1],
        wordvec_dim=256,
        num_heads=2,
        num_layers=2,
        max_length=30
        )
optimizer = optim.Adam(transformer.parameters(), lr=0.0006)

# 체크포인트 로드
checkpoint_path = './1.pth'
transformer, optimizer, epoch, loss_history = load_checkpoint(transformer, optimizer, checkpoint_path)

# 모델을 평가 모드로 전환
transformer.eval()


def generate_caption(model, feature, data):
    sample_caption = model.sample(feature, max_length=30)
    sample_caption = decode_captions(sample_caption, data['idx_to_word'])[0]
    return sample_caption

from tqdm import tqdm

# 예측 수행
student_id = "2020047029"  # FIXME
pred = []
nice_feat = data['nice_feature']
nice_feat = np.expand_dims(nice_feat, axis=1)

for i in tqdm(range(len(nice_feat))):
# for i in tqdm(range(1000)):
    caption = generate_caption(transformer, nice_feat[i], data)
    image_id = i + 1
    pred.append({'image_id': image_id, 'caption': caption})

result = {"student_id": student_id, "prediction": pred}
json.dump(result, open('pred.json', 'w'))

print("Predictions saved to pred.json")

base dir  /content/drive/MyDrive/cvhw3/CV/datasets/coco_captioning
Checkpoint loaded: epoch 1


100%|██████████| 1000/1000 [13:09<00:00,  1.27it/s]

Predictions saved to pred.json


In [ ]:
import torch
import torch.optim as optim

def load_checkpoint(model, optimizer, file_path):
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss_history = checkpoint['loss_history']
    print(f"Checkpoint loaded: epoch {epoch}")
    return model, optimizer, epoch, loss_history

data = load_coco_data(pca_features=False)
torch.manual_seed(231)
np.random.seed(231)

# 모델과 옵티마이저 초기화
transformer = CaptioningTransformer(
        word_to_idx=data['word_to_idx'],
        input_dim=data['train_features'].shape[1],
        wordvec_dim=256,
        num_heads=2,
        num_layers=2,
        max_length=30
        )
optimizer = optim.Adam(transformer.parameters(), lr=0.0006)

# 체크포인트 로드
checkpoint_path = './1.pth'
transformer, optimizer, epoch, loss_history = load_checkpoint(transformer, optimizer, checkpoint_path)

# 모델을 평가 모드로 전환
transformer.eval()

import heapq
import numpy as np

def beam_search(transformer, feature, beam_width=2, max_length=30):
    start_token = [data['word_to_idx']['<START>']]
    end_token = data['word_to_idx']['<END>']

    sequences = [[start_token, 0.0]]
    while len(sequences[0][0]) < max_length:

        all_candidates = []
        for seq, score in sequences:
            #print(seq)
            if seq[-1] == end_token:
                all_candidates.append((seq, score))
                continue

            partial_caption = torch.LongTensor([seq])
            predictions = transformer.sample(feature, partial_caption)
            predictions = predictions.squeeze(0)

            for idx, p in enumerate(predictions):

                p = max(p.item(), 1e-8)
                candidate = [seq + [idx], score - np.log(p)]
                all_candidates.append(candidate)

            ordered = sorted(all_candidates, key=lambda tup: tup[1])
            ordered = [tup for tup in ordered if 3 not in tup[0]]

            #print(ordered)

            sequences = ordered[:beam_width] # UNK gone

        if all(seq[-1] == end_token for seq, score in sequences):
            break

    #print(f"sequences : {sequences}")
    return sequences[0][0]


def generate_caption(feature):
    sample_caption = beam_search(transformer, feature)
    #print(sample_caption)
    sample_caption = decode_captions(np.array(sample_caption), data['idx_to_word'])
    return sample_caption

from tqdm import tqdm


student_id = "2020047029"  # FIXME
pred = []
nice_feat = data['nice_feature']
nice_feat = np.expand_dims(nice_feat, axis=1)

for i in tqdm(range(1000)):
    caption = generate_caption(nice_feat[i])
    image_id = i + 1
    pred.append({'image_id' : image_id, 'caption' : caption})

result = {"student_id" : student_id, "prediction" : pred}
json.dump(result, open('pred_beamtrue.json', 'w'))

base dir  /content/drive/MyDrive/cvhw3/CV/datasets/coco_captioning
Checkpoint loaded: epoch 1


  0%|          | 0/1000 [00:00<?, ?it/s]

torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])


  0%|          | 1/1000 [00:15<4:15:36, 15.35s/it]

torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])


...

torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])


  1%|          | 9/1000 [02:49<5:05:14, 18.48s/it]

torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])
torch.Size([1, 50261])


  1%|          | 9/1000 [03:05<5:41:18, 20.66s/it]


KeyboardInterrupt: 

In [ ]:
# Load COCO data from disk into a dictionary.
data = load_coco_data(pca_features=False)

# Print out all the keys and values from the data dictionary.
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

base dir  /content/drive/MyDrive/cvhw3/CV/datasets/coco_captioning


KeyboardInterrupt: 

# Transformer
As you have seen, RNNs are incredibly powerful but often slow to train. Further, RNNs struggle to encode long-range dependencies (though LSTMs are one way of mitigating the issue). In 2017, Vaswani et al introduced the Transformer in their paper ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) to a) introduce parallelism and b) allow models to learn long-range dependencies. The paper not only led to famous models like BERT and GPT in the natural language processing community, but also an explosion of interest across fields, including vision. While here we introduce the model in the context of image captioning, the idea of attention itself is much more general.



# Transformer: Multi-Headed Attention

### Dot-Product Attention

Recall that attention can be viewed as an operation on a query $q\in\mathbb{R}^d$, a set of value vectors $\{v_1,\dots,v_n\}, v_i\in\mathbb{R}^d$, and a set of key vectors $\{k_1,\dots,k_n\}, k_i \in \mathbb{R}^d$, specified as

\begin{align}
c = \sum_{i=1}^{n} v_i \alpha_i &\alpha_i = \frac{\exp(k_i^\top q)}{\sum_{j=1}^{n} \exp(k_j^\top q)} \\
\end{align}

where $\alpha_i$ are frequently called the "attention weights", and the output $c\in\mathbb{R}^d$ is a correspondingly weighted average over the value vectors.

### Self-Attention
In Transformers, we perform self-attention, which means that the values, keys and query are derived from the input $X \in \mathbb{R}^{\ell \times d}$, where $\ell$ is our sequence length. Specifically, we learn parameter matrices $V,K,Q \in \mathbb{R}^{d\times d}$ to map our input $X$ as follows:

\begin{align}
v_i = Vx_i\ \ i \in \{1,\dots,\ell\}\\
k_i = Kx_i\ \ i \in \{1,\dots,\ell\}\\
q_i = Qx_i\ \ i \in \{1,\dots,\ell\}
\end{align}

### Multi-Headed Scaled Dot-Product Attention
In the case of multi-headed attention, we learn a parameter matrix for each head, which gives the model more expressivity to attend to different parts of the input. Let $h$ be number of heads, and $Y_i$ be the attention output of head $i$. Thus we learn individual matrices $Q_i$, $K_i$ and $V_i$. To keep our overall computation the same as the single-headed case, we choose $Q_i \in \mathbb{R}^{d\times d/h}$, $K_i \in \mathbb{R}^{d\times d/h}$ and $V_i \in \mathbb{R}^{d\times d/h}$. Adding in a scaling term $\frac{1}{\sqrt{d/h}}$ to our simple dot-product attention above, we have

\begin{equation} \label{qkv_eqn}
Y_i = \text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)(XV_i)
\end{equation}

where $Y_i\in\mathbb{R}^{\ell \times d/h}$, where $\ell$ is our sequence length.

In our implementation, we apply dropout to the attention weights (though in practice it could be used at any step):

\begin{equation} \label{qkvdropout_eqn}
Y_i = \text{dropout}\bigg(\text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)\bigg)(XV_i)
\end{equation}

Finally, then the output of the self-attention is a linear transformation of the concatenation of the heads:

\begin{equation}
Y = [Y_1;\dots;Y_h]A
\end{equation}

were $A \in\mathbb{R}^{d\times d}$ and $[Y_1;\dots;Y_h]\in\mathbb{R}^{\ell \times d}$.

Implement multi-headed scaled dot-product attention in the `MultiHeadAttention` class in the file `CV/transformer_layers.py`. The code below will check your implementation. The relative error should be less than `e-3`.

In [ ]:
torch.manual_seed(231)

# Choose dimensions such that they are all unique for easier debugging:
# Specifically, the following values correspond to N=1, H=2, T=3, E//H=4, and E=8.
batch_size = 1
sequence_length = 3
embed_dim = 8
attn = MultiHeadAttention(embed_dim, num_heads=2)

# Self-attention.
data = torch.randn(batch_size, sequence_length, embed_dim)
self_attn_output = attn(query=data, key=data, value=data)

# Masked self-attention.
mask = torch.randn(sequence_length, sequence_length) < 0.5
masked_self_attn_output = attn(query=data, key=data, value=data, attn_mask=mask)

# Attention using two inputs.
other_data = torch.randn(batch_size, sequence_length, embed_dim)
attn_output = attn(query=data, key=other_data, value=other_data)

expected_self_attn_output = np.asarray([[
[-0.2494,  0.1396,  0.4323, -0.2411, -0.1547,  0.2329, -0.1936,
          -0.1444],
         [-0.1997,  0.1746,  0.7377, -0.3549, -0.2657,  0.2693, -0.2541,
          -0.2476],
         [-0.0625,  0.1503,  0.7572, -0.3974, -0.1681,  0.2168, -0.2478,
          -0.3038]]])

expected_masked_self_attn_output = np.asarray([[
[-0.1347,  0.1934,  0.8628, -0.4903, -0.2614,  0.2798, -0.2586,
          -0.3019],
         [-0.1013,  0.3111,  0.5783, -0.3248, -0.3842,  0.1482, -0.3628,
          -0.1496],
         [-0.2071,  0.1669,  0.7097, -0.3152, -0.3136,  0.2520, -0.2774,
          -0.2208]]])

expected_attn_output = np.asarray([[
[-0.1980,  0.4083,  0.1968, -0.3477,  0.0321,  0.4258, -0.8972,
          -0.2744],
         [-0.1603,  0.4155,  0.2295, -0.3485, -0.0341,  0.3929, -0.8248,
          -0.2767],
         [-0.0908,  0.4113,  0.3017, -0.3539, -0.1020,  0.3784, -0.7189,
          -0.2912]]])

print('self_attn_output error: ', rel_error(expected_self_attn_output, self_attn_output.detach().numpy()))
print('masked_self_attn_output error: ', rel_error(expected_masked_self_attn_output, masked_self_attn_output.detach().numpy()))
print('attn_output error: ', rel_error(expected_attn_output, attn_output.detach().numpy()))

self_attn_output error:  0.0003775124598178026
masked_self_attn_output error:  0.0001526367643724865
attn_output error:  0.0003527921483788199


# Positional Encoding

While transformers are able to easily attend to any part of their input, the attention mechanism has no concept of token order. However, for many tasks (especially natural language processing), relative token order is very important. To recover this, the authors add a positional encoding to the embeddings of individual word tokens.

Let us define a matrix $P \in \mathbb{R}^{l\times d}$, where $P_{ij} = $

$$
\begin{cases}
\text{sin}\left(i \cdot 10000^{-\frac{j}{d}}\right) & \text{if j is even} \\
\text{cos}\left(i \cdot 10000^{-\frac{(j-1)}{d}}\right) & \text{otherwise} \\
\end{cases}
$$

Rather than directly passing an input $X \in \mathbb{R}^{l\times d}$ to our network, we instead pass $X + P$.

Implement this layer in `PositionalEncoding` in `CV/transformer_layers.py`. Once you are done, run the following to perform a simple test of your implementation. You should see errors on the order of `e-3` or less.

In [ ]:
torch.manual_seed(231)

batch_size = 1
sequence_length = 2
embed_dim = 6
data = torch.randn(batch_size, sequence_length, embed_dim)

pos_encoder = PositionalEncoding(embed_dim)
output = pos_encoder(data)

expected_pe_output = np.asarray([[[-1.2340,  1.1127,  1.6978, -0.0865, -0.0000,  1.2728],
                                  [ 0.9028, -0.4781,  0.5535,  0.8133,  1.2644,  1.7034]]])

print('pe_output error: ', rel_error(expected_pe_output, output.detach().numpy()))

pe_output error:  0.00010421011374914356


# Transformer for Image Captioning
Now that you have implemented the previous layers, you can combine them to build a Transformer-based image captioning model. Open the file `CV/classifiers/transformer.py` and look at the `CaptioningTransformer` class.

Implement the `forward` function of the class. After doing so, run the following to check your forward pass using a small test case; you should see error on the order of `e-5` or less.

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

N, D, W = 4, 20, 30
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 3

transformer = CaptioningTransformer(
    word_to_idx,
    input_dim=D,
    wordvec_dim=W,
    num_heads=2,
    num_layers=2,
    max_length=30
)

# Set all model parameters to fixed values
for p in transformer.parameters():
    p.data = torch.tensor(np.linspace(-1.4, 1.3, num=p.numel()).reshape(*p.shape))

features = torch.tensor(np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D))
captions = torch.tensor((np.arange(N * T) % V).reshape(N, T))

scores = transformer(features, captions)
expected_scores = np.asarray([[[-16.9532,   4.8261,  26.6054],
         [-17.1033,   4.6906,  26.4844],
         [-15.0708,   4.1108,  23.2924]],
        [[-17.1767,   4.5897,  26.3562],
         [-15.6017,   4.8693,  25.3403],
         [-15.1028,   4.6905,  24.4839]],
        [[-17.2172,   4.7701,  26.7574],
         [-16.6755,   4.8500,  26.3754],
         [-17.2172,   4.7701,  26.7574]],
        [[-16.3669,   4.1602,  24.6872],
         [-16.7897,   4.3467,  25.4831],
         [-17.0103,   4.7775,  26.5652]]])
print('scores error: ', rel_error(expected_scores, scores.detach().numpy()))

scores error:  5.056720614439509e-06


# Overfit Transformer Captioning Model on Small Data
Run the following to overfit the Transformer-based captioning model on the same small dataset as we used for the RNN previously.

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

data = load_coco_data(max_train=50)

transformer = CaptioningTransformer(
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          wordvec_dim=256,
          num_heads=2,
          num_layers=2,
          max_length=30
        )



transformer_solver = CaptioningSolverTransformer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=110,
           batch_size=25,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

transformer_solver.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

base dir  /content/drive/MyDrive/cvhw3/CV/datasets/coco_captioning
(Iteration 1 / 220) loss: 10.790522
(Epoch 1 / 110) validation loss: 8.673990
Checkpoint saved at epoch 1
(Epoch 2 / 110) validation loss: 8.288123
Checkpoint saved at epoch 2
(Epoch 3 / 110) validation loss: 7.562152
Checkpoint saved at epoch 3
(Epoch 4 / 110) validation loss: 6.996141
Checkpoint saved at epoch 4
(Epoch 5 / 110) validation loss: 6.575783
Checkpoint saved at epoch 5
(Iteration 11 / 220) loss: 5.238126
(Epoch 6 / 110) validation loss: 6.084833
Checkpoint saved at epoch 6
(Epoch 7 / 110) validation loss: 5.627957


KeyboardInterrupt: 

Print final training loss. You should see a final loss of less than 0.03.

In [ ]:
print('Final loss: ', transformer_solver.loss_history[-1])

# Transformer Sampling at Test Time
The sampling code has been written for you. You can simply run the following to compare with the previous results with the RNN. As before the training results should be much better than the validation set results, given how little data we trained on.

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = transformer.sample(features, max_length=30)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()

# Train Transformer Captioning Model on entire datasets.
This might take while.


In [ ]:
torch.manual_seed(231)
np.random.seed(231)

data = load_coco_data(pca_features=False)

base dir  /content/drive/MyDrive/cvhw3/CV/datasets/coco_captioning


In [ ]:


transformer = CaptioningTransformer(
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          wordvec_dim=256,
          num_heads=2,
          num_layers=2,
          max_length=30
        )

transformer = transformer.to(torch.device("cuda:0" if torch.cuda.is_available() == True else "cpu"))


transformer_solver = CaptioningSolverTransformer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=5,
           batch_size=256,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

transformer_solver.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

(Iteration 1 / 7350) loss: 10.877545
(Iteration 11 / 7350) loss: 5.881330
(Iteration 21 / 7350) loss: 4.548771
(Iteration 31 / 7350) loss: 4.260265
(Iteration 41 / 7350) loss: 4.127852
(Iteration 51 / 7350) loss: 3.971354
(Iteration 61 / 7350) loss: 3.885946
(Iteration 71 / 7350) loss: 3.706767
(Iteration 81 / 7350) loss: 3.663498
(Iteration 91 / 7350) loss: 3.516140
(Iteration 101 / 7350) loss: 3.539637
(Iteration 111 / 7350) loss: 3.330622
(Iteration 121 / 7350) loss: 3.172125
(Iteration 131 / 7350) loss: 3.263559
(Iteration 141 / 7350) loss: 3.035350
(Iteration 151 / 7350) loss: 3.068617
(Iteration 161 / 7350) loss: 2.945641
(Iteration 171 / 7350) loss: 2.861845
(Iteration 181 / 7350) loss: 2.944737
(Iteration 191 / 7350) loss: 2.853827
(Iteration 201 / 7350) loss: 2.825539
(Iteration 211 / 7350) loss: 2.815242
(Iteration 221 / 7350) loss: 2.791818
(Iteration 231 / 7350) loss: 2.750729
(Iteration 241 / 7350) loss: 2.629654
(Iteration 251 / 7350) loss: 2.764151
(Iteration 261 / 7350)

# Transformer Sampling at Test Time
The sampling code has been written for you. You can simply run the following to compare with the previous results.

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = transformer.sample(features, max_length=30)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()

# Try your model on test set.


In [ ]:
import json
import os
from tqdm import tqdm

# def generate_caption(feature):
#     # FIXME
#     return "DUMMY CAPTION"

# student_id = "0000000000" # FIXME
# pred = []
# nice_feat = data['nice_feature']
# nice_feat = np.expand_dims(nice_feat, axis=1)

# for i in tqdm(range(len(nice_feat))):
#     caption = generate_caption(nice_feat[i])
#     image_id = i + 1
#     pred.append({'image_id' : image_id, 'caption' : caption})

# result = {"student_id" : student_id, "prediction" : pred}
# json.dump(result, open('pred.json', 'w'))

def generate_caption(model, feature, data):
    sample_caption = model.sample(feature, max_length=30)
    sample_caption = decode_captions(sample_caption, data['idx_to_word'])[0]
    return sample_caption


# 예측 수행
student_id = "2020047029"  # FIXME
pred = []
nice_feat = data['nice_feature']
nice_feat = np.expand_dims(nice_feat, axis=1)

# for i in tqdm(range(len(nice_feat))):
for i in tqdm(range(1000)):
    caption = generate_caption(transformer, nice_feat[i], data)
    image_id = i + 1
    pred.append({'image_id': image_id, 'caption': caption})

result = {"student_id": student_id, "prediction": pred}
json.dump(result, open('pred_beamfalse.json', 'w'))

print("Predictions saved to pred.json")